#### Note: This code is written for Preprocessing Twitter data files extracted using stweet. For other extraction files, some modifications are required 

For preprocessing on multiple files, add the name of the csv files in list named -- > files_list (Declared towards the end) 
Modify the paths according to your local folder paths 

Installations

In [48]:
!pip install stanza
!pip install unidecode
!pip install contractions
!pip install nltk
!python -m spacy download en_core_web_sm
!pip install emoji
!pip install emoticons_list
!pip install emot
!pip install autocorrect
!pip install langdetect
!pip install textblob
!pip install krovetzstemmer
!pip install swifter

     |████████████████████████████████| 13.6 MB 82 kB/s  eta 0:00:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
ERROR: Could not find a version that satisfies the requirement emoticons_list
ERROR: No matching distribution found for emoticons_list


     |████████████████████████████████| 293 kB 3.3 MB/s eta 0:00:01


Imports

In [29]:
import os 
from tqdm.auto import tqdm 
from datetime import datetime
import pandas as pd
import re
from autocorrect import Speller
from spacy_langdetect import LanguageDetector
import spacy
import unidecode
from pathlib import Path
import itertools
import contractions
import nltk
import stanza
from functools import reduce
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
import emoji
import emot 
from string import punctuation
from langdetect import detect_langs,detect
from textblob import TextBlob
from krovetzstemmer import Stemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import swifter

## Initialisations and general declarations

tqdm.pandas()
tokenizer = TweetTokenizer(preserve_case=False, 
                           strip_handles=True,
                           reduce_len=True)
nltk.download('stopwords')
nltk.download('punkt')
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')
spell = Speller(lang = 'en')
emot_obj = emot.core.emot()



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azulcou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/azulcou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


File Paths 

In [30]:
#file paths


slang_file_path = r"/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/data/task-2-preprocessing/mapping/slang.txt"
folder_path = Path(r'/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/data/task-1-data-scraping/Twitter')
output_folder_path = Path(r'/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/data/task-2-preprocessing/preprocessed_raw_files/Twitter')

In [31]:

os.getcwd()

'/mnt/c/Users/ksafa/OneDrive/Desktop/Omdena/omdena-colombia-socialnetwork/src/tasks/task-2-preprocessing'

In [32]:
files_list = ['ColombiaSOS_SPANISH_tweets_stweet.csv',
              'soscolombia_SPANISH_tweets_stweet.csv']

In [33]:
#Checking if columns are similar
for file in files_list:
    data = pd.read_csv(folder_path/file)
    print(data.columns)
    

Index(['created_at', 'id_str', 'conversation_id_str', 'full_text', 'lang',
       'favorited', 'retweeted', 'retweet_count', 'favorite_count',
       'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media'],
      dtype='object')
Index(['created_at', 'id_str', 'conversation_id_str', 'full_text', 'lang',
       'favorited', 'retweeted', 'retweet_count', 'favorite_count',
       'reply_count', 'quote_count', 'quoted_status_id_str',
       'quoted_status_short_url', 'quoted_status_expand_url', 'user_id_str',
       'user_name', 'user_full_name', 'user_verified',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'hashtags',
       'mentions', 'urls', 'media'],
      dtype='object')


In [34]:
#Slang dictionary 

f = open(slang_file_path, 'r')
slang_dict = {}
for line in f:
    k, v = line.strip().split('=')
    slang_dict[k.strip()] = v.strip()

f.close()

print(slang_dict)

{'AFAIK': 'As Far As I Know', 'AFK': 'Away From Keyboard', 'ASAP': 'As Soon As Possible', 'ATK': 'At The Keyboard', 'ATM': 'At The Moment', 'A3': 'Anytime, Anywhere, Anyplace', 'BAK': 'Back At Keyboard', 'BBL': 'Be Back Later', 'BBS': 'Be Back Soon', 'BFN': 'Bye For Now', 'B4N': 'Bye For Now', 'BRB': 'Be Right Back', 'BRT': 'Be Right There', 'BTW': 'By The Way', 'B4': 'Before', 'CU': 'See You', 'CUL8R': 'See You Later', 'CYA': 'See You', 'FAQ': 'Frequently Asked Questions', 'FC': 'Fingers Crossed', 'FWIW': "For What It's Worth", 'FYI': 'For Your Information', 'GAL': 'Get A Life', 'GG': 'Good Game', 'GN': 'Good Night', 'GMTA': 'Great Minds Think Alike', 'GR8': 'Great!', 'G9': 'Genius', 'IC': 'I See', 'ICQ': 'I Seek you (also a chat program)', 'ILU': 'ILU: I Love You', 'IMHO': 'In My Honest/Humble Opinion', 'IMO': 'In My Opinion', 'IOW': 'In Other Words', 'IRL': 'In Real Life', 'KISS': 'Keep It Simple, Stupid', 'LDR': 'Long Distance Relationship', 'LMAO': 'Laugh My A.. Off', 'LOL': 'Laug

In [35]:
data.head()

,created_at,id_str,conversation_id_str,full_text,lang,favorited,retweeted,retweet_count,favorite_count,reply_count,...,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media
0,2021-04-04T17:47:58+00:00,1378766306112069636,1378766306112069636,Ver noticias en este país es ver 5 o 6 noticia...,es,False,False,1,2,0,...,1235059598136090624,dannaso18864244,bailandin _Sofia,False,NaN,NaN,"#SOSColombia , #Colombia",NaN,NaN,[]
1,2021-04-04T16:29:41+00:00,1378746605764026372,1378746605764026372,#25 masacres en 2021 en Colombia 🇨🇴 #SOSColomb...,es,False,False,0,0,0,...,90917823,FatimaMG,Fátima,False,NaN,NaN,#SOSColombia,NaN,NaN,[]
2,2021-04-04T15:56:48+00:00,1378738332021231618,1378733935430139907,@angelamrobledo @IvanDuque La #doblemoral y co...,es,False,False,0,1,0,...,126344207,Vlasoga,wlasoga,False,1.378734e+18,1.657483e+08,"#doblemoral , #SOSColombia","angelamrobledo , IvanDuque",NaN,"[{""url"": ""https://pbs.twimg.com/media/EyJDNJ9W..."
3,2021-04-04T15:39:36+00:00,1378734004057354249,1378734004057354249,Corrupción en órganos de control manejados por...,es,False,False,2,1,0,...,80101459,maomontenegro,Animales Seres Sintientes y Conscientes. Avanc...,False,NaN,NaN,"#SOSColombia , #SOSColombis , #AntiUribistaSig...",RamiroBejaranoG,https://t.co/MRMC70aXED,[]
4,2021-04-04T15:33:35+00:00,1378732486910488582,1369122514899763201,@confispcol @WRadioColombia @lafm @GustavoBoli...,es,False,False,2,1,0,...,80101459,maomontenegro,Animales Seres Sintientes y Conscientes. Avanc...,False,1.369123e+18,1.090272e+18,"#SOSColombis , #AntiUribistaSigueAntiUribista ...","confispcol , WRadioColombia , lafm , GustavoBo...",NaN,[]


In [36]:
data.full_text

0        Ver noticias en este país es ver 5 o 6 noticia...
1        #25 masacres en 2021 en Colombia 🇨🇴 #SOSColomb...
2        @angelamrobledo @IvanDuque La #doblemoral y co...
3        Corrupción en órganos de control manejados por...
4        @confispcol @WRadioColombia @lafm @GustavoBoli...
                               ...                        
43398    HASTA QUE LA DIGNIDAD SE HAGA COSTUMBRE\r\nEst...
43399    NO IMPORTA LO QUE HAGAN O DIGAN\r\nNo nos vamo...
43400    Ya no son 60 días son casi 120 días\r\nEl terr...
43401    Un dato ahí:\r\n\r\nEn lo que va del año 2021,...
43402    Para q tenga Biden callado sobre la violación ...
Name: full_text, Length: 43403, dtype: object

In [37]:
data.full_text[0]

'Ver noticias en este país es ver 5 o 6 noticias seguidas con asesinatos, masacres y guerra en diferentes partes del país ! \r\nUn país que ruega paz ... #SOSColombia #Colombia'

# Detecting Language 
 

In [38]:
#Using TextBlob

# lang = TextBlob(data.full_text[0])
# print(lang.detect_language())


In [39]:
#Using langdetect

print(detect_langs(data.full_text[0]))
print(detect(data.full_text[0]))

[es:0.9999942377223618]
es


Helper Functions

In [40]:
stanza.download('es')
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')


2021-09-04 19:27:58 INFO: Downloading default packages for language: es (Spanish)...


2021-09-04 19:28:03 INFO: File exists: /home/azulcou/stanza_resources/es/default.zip.
2021-09-04 19:28:13 INFO: Finished downloading models and saved to /home/azulcou/stanza_resources.
2021-09-04 19:28:13 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

2021-09-04 19:28:13 INFO: Use device: cpu
2021-09-04 19:28:13 INFO: Loading: tokenize
2021-09-04 19:28:13 INFO: Loading: mwt
2021-09-04 19:28:13 INFO: Loading: pos
2021-09-04 19:28:14 INFO: Loading: lemma
2021-09-04 19:28:14 INFO: Done loading processors!


In [46]:
def language_detect(text):
    ##Using langdetect
    detect(text)
#     print('lang')

def lower_case_conversion(text):
#     print('lower')
    
    return text.lower()


def removing_html_tag(text):
#     print('html')
    clean_text = re.compile('<.*?>')
    return re.sub(clean_text,'',text)


def removing_urls(text):
#     print('url')
    return re.sub(r'https?://\S+|www\.\S+','',text)

def spelling_correction_en(text):
#     print('spell')
#     spell = Speller(lang = 'en')
    return spell(text)

def accented_to_ascii(text):
#     print('accented')
    return unidecode.unidecode(text)

def short_to_original(text):
#     print('short')
    #replace the slang word with meaning
    
    '''
    f = open(slang_file_path, 'r')
    slang_dict = {}
    for line in f:
        k, v = line.strip().split('=')
        slang_dict[k.strip()] = v.strip()

    f.close()
    '''
    text_tokens=text.split()
    for i,word in enumerate(text_tokens):
        if word in slang_dict.keys():
            text_tokens[i]=slang_dict[word]

    text_new=" ".join(text_tokens)

    return text_new

def standardizing(text):
#     print('standard')
    #One letter in a word should not be present more than twice in continuation
    return ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))


def contractions_text(text):
#     print('contractions')
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shortened words
      expanded_words.append(contractions.fix(word))   

    return ' '.join(expanded_words)

def stemming(text):
    return [stemmer.stem(token) for token in text.split()]
    

def lemmatization(text):
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

def spanish(text):
    return " ".join(w for w  in nltk.wordpunct_tokenize(text) \
         if w.lower() not in words or not w.isalpha())
    
def extract_emojis(text):
#     print('extract emoji')
    return [''.join(c for c in text if c in emoji.UNICODE_EMOJI['en'])]

def extract_emoticons(text):
#     print('emoticons')
    
    return emot_obj.emoticons(text)['value']


def remove_punctuation(text):
#     print('punc')
    return text.translate(str.maketrans('', '', punctuation))

def remove_emojis(text):
#     print('emoji')
    return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI['en'])

def remove_stopwords(text):
#     print('stop')
#     nltk.download('stopwords')
    stop = set(stopwords.words('spanish'))
    return ' '.join([word for word in text.split() if word.lower() not in stop])

    
def remove_extra_spaces(text):
#     print('space')
    space_pattern = r'\s+'
    without_space = re.sub(pattern=space_pattern, repl=" ", string=text)
    return without_space

def remove_single_char(text):
#     print('single')
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc
    
def twitter_preprocessed_date(text):
    return datetime.strptime(text.split('T')[0], "%Y-%m-%d").date()

def remove_mention_text(text):
#     print('mention')
    mention_pattern = r'@\S+'
    return re.sub(pattern=mention_pattern, repl=' ', string=text)
    
def remove_digits(text):
#     print('digit')
    digits = r'[0-9]'
    return re.sub(pattern=digits, repl=' ', string=text)
    

In [25]:
    
def spanish_preprocessing(text, func_list):
    
    return reduce(lambda o, func: func(o), func_list, text)

In [26]:
preprocessed_data = data.copy()

## Storing emoticons and emojis separately

preprocessed_data['emoji_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emojis(x))
                                                                       
preprocessed_data['emoticons_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emoticons(removing_urls(x)))

preprocessed_data['preprocessed_created_at'] = preprocessed_data['created_at'].apply(lambda x: twitter_preprocessed_date(x))


In [17]:
print(preprocessed_data['emoji_list'].astype(str).unique())

["['']" "['🐷']" "['🌎🌍🌍🐷👇👇']" "['🌍']" "['📌']" "['🐷🌎']" "['🐷🐷👇👇']"
 "['🐷🐷😭😭']" "['🐷👇👇👇']" "['🐷🐷👇🐷👇👇👇']" "['🌎🐷👇']" "['🎶🎶🎶🎵🙈🙉🙊']" "['✊🏼']"
 "['‼']" "['🍑']" "['🙏']" "['😑']" "['🤦🏽♂🤦🏽♂🤦🏽♂']" "['➕']" "['🤣🤣🤣🤣']"
 "['🤣']" "['😞']" "['🆘🖕💩']" "['😔']" "['😒😒🤫🤔😒😒😎😎😎✊✊✊']" "['💩']"
 "['✊✊✊✊😎😎😎👏👏👏💖💖🥺🥺']" "['👁👏🏽👇🏽']" "['🤮🤮']" "['🤔😘']" "['🤔🤣😂']" "['💪']"
 "['⭕']" "['🧵🤫']" "['✊✊✊']" "['🤣🤣🤣']" "['🌎']" "['😁😁😁❤❤']" "['☠🎭']" "['😢']"
 "['😭🙏']" "['😫']" "['😂😂']" "['🔗']" "['⬇']" "['👇']" "['🔥']" "['🤝']"
 "['😂😂😂']" "['💪✊']" "['😂😂💪']" "['❓']" "['🔫🤬']" "['▶']" "['👇👇👇👇👇👇']"
 "['🤦♀💔']" "['💯💪']" "['💪😍']" "['🤮🤮🤮']" "['😳😐']" "['🤬😡']" "['💩💩']"
 "['😓😠😫']" "['🤮']" "['🗣']" "['✌💛💙❤']" "['🖐🏾🖐🏻']" "['😪✊🏼✊✊🏽✊🏾✊🏻✊🏿']"
 "['🤮💩']" "['☠']" "['👉']" "['✨💋']" "['🤔🙄🤦🏻♀😡']" "['🤌🏻🤦🏼♂']" "['📢📢']"
 "['👇👇']" "['👇👇👇']" "['💩💪']" "['🎥']" "['💔😪🙏🏽🌿']" "['🆘']" "['🙄🙏🙏']" "['💔']"
 "['😡😡😡😭😭😭😓😓😓😓']" "['😪']" "['😔💔']" "['🤬']" "['🎗💛💙🖤']" "['🔴']" "['🌎🐷']"
 "['🐷🐷👇']" "['🐷😭😭👇👇👇👇👇👇👇👇👇']" "['🐷👇']" "['😡']" "['💪💪💪🐷']" "['🐷👇👇👇👇']"
 "['😠']" "['💀🦴']" "['☝']" "['✊✊🙏😱😱✊😡

In [18]:
print(preprocessed_data['emoticons_list'].astype(str).unique())

['[]' "[':#']" "[':P']" "[':L']" "['oO']" "['XP']" "['XD']" "['D:']"
 "[':3']" "[':@']"]


In [19]:
##Stemming
spanish_func_list_stem  = [short_to_original, contractions_text, 
             standardizing, lower_case_conversion, 
             remove_emojis,remove_mention_text, removing_html_tag, 
             removing_urls, remove_punctuation, remove_extra_spaces,
             remove_digits, remove_single_char,lower_case_conversion, accented_to_ascii,
                      remove_stopwords,
                    stemming] 
preprocessed_data['full_text'][:2].progress_apply(lambda x: spanish_preprocessing(x, spanish_func_list_stem))

0    [ver, notici, pais, ver, notici, segu, asesina...
1                 [masacr, colombi, soscolombi, dolor]
Name: full_text, dtype: object

In [20]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [47]:
##Lemmatization
spanish_func_list_lemma  = [short_to_original, contractions_text, 
             standardizing, lower_case_conversion, 
             remove_emojis,remove_mention_text, removing_html_tag, 
             removing_urls, remove_punctuation, remove_extra_spaces,
             remove_digits, remove_single_char,lower_case_conversion, accented_to_ascii,
                      remove_stopwords,
                    lemmatization] 
preprocessed_data['full_text'][:2].progress_apply(lambda x: spanish_preprocessing(x, spanish_func_list_lemma))

0    [ver, noticia, paar, ver, noticia, seguido, as...
1             [masacre, colombio, soscolombiar, dolor]
Name: full_text, dtype: object

In [22]:
from multiprocessing import  Pool

In [23]:
import numpy as np

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
def preprocessed_data_column(df):
    df['preprocessed_data'] = df['full_text'].apply(lambda x: spanish_preprocessing(x, spanish_func_list_lemma))
    

In [ ]:

preprocessed_data_parallel = parallelize_dataframe(preprocessed_data[:2], preprocessed_data_column)

In [ ]:
'''
This cells takes a long time to run. You can run the preprocessing on multiple files
with the next code block. Suggest to SKIP THIS. 
Run only if exploring (it can take hours to run!) 
'''

# preprocessed_data['preprocessed_data'] = preprocessed_data['full_text'].progress_apply(lambda x: spanish_preprocessing(x, spanish_func_list_lemma))

# preprocessed_data 

# print(f'Original Data ---> {preprocessed_data.full_text[0]}\n \n')
# print(f'Preprocessed Data ---> {preprocessed_data.preprocessed_data[0]}')

# print('Lemmatization output')
# print(f'Original Data ---> {preprocessed_data.full_text[0]}\n \n')
# print(f'Preprocessed Data ---> {preprocessed_data.preprocessed_data[0]}')

# print(f'Original Data ---> {preprocessed_data.full_text[1]}\n \n')
# print(f'Preprocessed Data ---> {preprocessed_data.preprocessed_data[1]}')

# print('Lemmatization')
# print(f'Original Data ---> {preprocessed_data.full_text[1]}\n \n')
# print(f'Preprocessed Data ---> {preprocessed_data.preprocessed_data[1]}')

## Preprocessing Multiple Files together 

Modify files_list based on the name of csv files 

In [ ]:
files_list = ['VivaElParoNacional_SPANISH_tweets_stweet.csv',
              'comitedelparo_SPANISH_tweets_stweet.csv',
              'noalareformatributaria _SPANISH_tweets_stweet.csv',
              'soscolombia_SPANISH_tweets_stweet.csv',
              'soscolombiaendictadura_SPANISH_tweets_stweet.csv',
              'soscolombianoduerme_SPANISH_tweets_stweet.csv',
              'ColombiaSOS_SPANISH_tweets_stweet.csv'
             ]


In [ ]:
#Checking if columns are similar
for file in files_list:
    data = pd.read_csv(folder_path/file)
    preprocessed_data = data.copy()

    ## Storing emoticons and emojis separately

    preprocessed_data['emoji_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emojis(x))

    preprocessed_data['emoticons_list'] = preprocessed_data['full_text'].apply(lambda x: extract_emoticons(x))

    preprocessed_data['preprocessed_created_at'] = preprocessed_data['created_at'].apply(lambda x: twitter_preprocessed_date(x))

    preprocessed_data['preprocessed_data'] = preprocessed_data['full_text'].progress_apply(lambda x: spanish_preprocessing(x, spanish_func_list_stem))

    preprocessed_data.to_csv(output_folder_path/file)